# Statistická práce

## Použitá data

Jako dataset používám dataset "ilc_di08" z Eurostatu, který obsahuje informace o mediánovém a průměrném ročním příjmu v závislosti na:
- zemi (evropské státy)
- roce (2005 - 2023)
- úrovni vzdělání (ED0-2, ED3-4, ED5-8 podle ISCED)
- věkové skupině (plánuji se zaměřit na 18–64 let)
- pohlaví

Státy:
- `CZ` - Česká republika
- `DE` - Německo
- `EU` - Evropa, průměr
- `UK` - Velká Británie
- `CH` - Švýčarsko
- `SE` - Serbsko
- `HU` - Maďarsko
- etc.

Úrovně vzdělání:
- `ED0-2` - Early childhood education - Lower secondary education
- `ED3_4` - Upper secondary education - Post-secondary non-tertiary education
- `ED5-8` - Short-cycle tertiary education - Doctoral or equivalent level

Věkové skupiny:
- `Y18-64` - od 18 do 64 let
- `Y_GE18` - od 18 let
- `Y_GE65` - od 65 let

Pohlaví:
- `F` - ženy
- `M` - muži
- `T` - všichni

Příjem:
- `MED_E` - průměrný
- `MEI_E` - mediánový

Měny:
- `EUR` - eura
- `NAC` - měna zemi
- `PPS` - pps